In [5]:
import pandas as pd
import os
import json
import tiktoken
import openai
import numpy as np
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

In [19]:
BASE_URL_4 = "https://danijel-test02.openai.azure.com/"
API_KEY_4 = "05fe89f89e984fd188cb6143d6f5699a"
DEPLOYMENT_NAME_4 = "gpt4"
model_4 = AzureChatOpenAI(
    openai_api_base=BASE_URL_4,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME_4,
    openai_api_key=API_KEY_4,
    openai_api_type="azure",
    temperature=0.0,
)

In [6]:
BASE_URL = "https://danijel-test01.openai.azure.com/"
API_KEY = "efd76f5c1a2a44beb3399c73c7d5695f"
DEPLOYMENT_NAME = "gpt-35-turbo"
model_3 = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
    temperature=0.0,
)

In [7]:
def gpt3_call(prompt):
    return model_3([HumanMessage(content=prompt)])

# Then you can use it like this:
# translation = gpt4_call("Pozdravljen svet!")

In [20]:
def gpt4_call(prompt):
    return model_4([HumanMessage(content=prompt)])

# Then you can use it like this:
# translation = gpt4_call("Pozdravljen svet!")

## FIND THE MAIN QUESTION AND KEYWORDS FROM EMAIL 📬📨

In [6]:
question =  f"""Spoštovani,
oče, Anton Kokalj, rojen 26.10.1936, je pri vas sklenil zavarovalno polico za investicijsko zavarovanje fleks za odrasle, št. police 8229870, sedaj bi želel dvigniti sredstva v obliki predujma. Zanima me kje najdem obrazec za dvig predujma.
Hvala in lep pozdrav,
Elizabeta Uršič.
"""

prompt_slo = f"""From the email below, find the main question and keywords.
Email:
Pozdravljeni, 
najlepša hvala za vaše obvestilo. Mi prosim lahko dopišete za katero zavarovanje pošiljate opomin. Lahko dopišete št. pogodbe, prosim. Pri vas imamo člani družine  zavarovano hišo, avto, zdravje, motor... 
Za odgovor se vam na najlepše zahvaljujem. 
Lep pozdrav
Vladimila Dobnik 
Output:
Vladimila Dobnik je vprašala, katero zavarovanje pošiljate opomin.
keywords -> "opomin, polica, premija, osebno zavarovanje, križanje polic, plačilo"

Email:
{question}
Output:
""" 

prompt_output = gpt4_call(prompt_slo).content

In [7]:
print(prompt_output)

Elizabeta Uršič je vprašala, kje najde obrazec za dvig predujma.
keywords -> "investicijsko zavarovanje, fleks, dvig predujma, obrazec"


In [8]:
question =  f"""Spoštovani.
V prilogi ,ki ste mi jo poslali žal nisem zasledil za katero zavarovanje gre.(Opomin zaradi neplačila zavarovalne premije št.: A-2023-0045509 Zavarovalec: 982068)
Lep pozdrav
PREŠEREN TONE
"""

prompt_eng = f"""From the email below, find the main question and keywords.
Email:
Pozdravljeni, 
najlepša hvala za vaše obvestilo. Mi prosim lahko dopišete za katero zavarovanje pošiljate opomin. Lahko dopišete št. pogodbe, prosim. Pri vas imamo člani družine  zavarovano hišo, avto, zdravje, motor... 
Za odgovor se vam na najlepše zahvaljujem. 
Lep pozdrav
Vladimila Dobnik 
Output:
Could you please let me know for which insurance you are sending the reminder? Can you write the number of the contracts, please.
keywords -> ["reminder", "policy", "premium", "personal insurance", "policy crossing", "payment"]

Email:
{question}
Output:
""" 

prompt_output = gpt4_call(prompt_eng).content

In [9]:
print(prompt_output)

Which insurance is the reminder for? (Reminder for non-payment of insurance premium no.: A-2023-0045509 Policyholder: 982068)
keywords -> ["reminder", "insurance", "non-payment", "premium", "policyholder"]


## FINDING BEST TAXONOMY FROM THE ONES THAT WE GOT FROM REDIS 📮🛢️

## FINDING BEST TAXONOMY FROM THE ONES THAT WE GOT FROM OPENAI USING MULTIPLE PROMPT 🤖⚙️

In [10]:
AI_found_taxonomies =  f"""["Ali je račun poravnan / plačan?", "Ali lahko sklenem nezgodno zavarovanje mladih med letom?", "Blokiran račun, urejanje terjatev", "Do kdaj lahko sklenem zavarovanje odpovedi turističnega zavarovanja?", "i.Triglav nivo zaupanja tier", "Informacije o opominu", "Kaj je digitalna poslovalnica i.triglav?", "Informacije o opominu"]"""

email_question_fact =  f"""I did not find which insurance the reminder is for in the attachment you sent me (Reminder for non-payment of insurance premium no.: A-2023-0045509 Policyholder: 982068). 
keywords -> ["reminder", "non-payment", "insurance premium", "attachment", "policyholder"]"""

judge_prompt = f"""Consider the following email and avaliable taxonomy classifications.
------
email:
{email_question_fact}
avaliable taxonomies:
{AI_found_taxonomies}
------
From the avaliable taxonomies, find the most relevant one for the email. The most relevant taxonomy is the one that best describes what happens in the email. Avaliable taxonomies are in a list and given in a random order.
""" 
prompt_output = gpt4_call(judge_prompt).content

In [11]:
print(prompt_output)

The most relevant taxonomy for the email is "Informacije o opominu".


In [62]:
AI_found_taxonomies =  f"""["Kako lahko prekličem dogovorjen termin s svetovalcem/zastopnikom?", "Blokiran račun, urejanje terjatev", "Do kdaj lahko sklenem zavarovanje odpovedi turističnega zavarovanja?","Investicijsko zavarovanje fleks za odrasle", "i.Triglav nivo zaupanja tier", "Informacije o opominu", "Kaj je digitalna poslovalnica i.triglav?", "Informacije o opominu"]"""

email_question_fact =  f"""Zadeva: Odpoved napovedanega obiska

Pozdravljeni!

Danes sem prejela vaš klic in najavo zav. zastopnika za jutri (18.4.) ob 19h na ime Anja Masten, Velike Žablje 60a, 5263 Dobravlje....
Ker je prišlo nekaj vmes in me ne bo doma, najavo odpovedujem. Ko bom zainteresirana, vas bom kontaktirala, zaenkrat me ne zanima.
Ker sem tudi klicala nazaj na vašo tel. številko in nisem dobila odgovora, vas prosim za povratno informacijo o odpovedi obiska.

Hvala in lep pozdrav!
Anja Masten
"""



judge_prompt_oneShot = f"""Consider the following text and avaliable taxonomy classifications.
From the avaliable taxonomies, find the most relevant one for the text. The most relevant taxonomy is the one that best describes the text. Avaliable taxonomies are in a list and given in a random order.
------
EXAMPLE:
text:
I did not find which insurance the reminder is for in the attachment you sent me (Reminder for non-payment of insurance premium no.: A-2023-0045509 Policyholder: 982068).
avaliable taxonomies:
["Informacije o opominu", "Blokiran račun, urejanje terjatev", "Kaj je digitalna poslovalnica i.triglav?", "Do kdaj lahko sklenem zavarovanje odpovedi turističnega zavarovanja?", "i.Triglav nivo zaupanja tier", "Ali lahko sklenem nezgodno zavarovanje mladih med letom?", "Ali je račun poravnan / plačan?", "Informacije o opominu"]
------
OUTPUT:
Informacije o opominu
------
REAL_PROMPT:
text:
{email_question_fact}
avaliable taxonomies:
{AI_found_taxonomies}
------
OUTPUT:
""" 
prompt_output = gpt4_call(judge_prompt_oneShot).content

In [63]:
print(prompt_output)

Investicijsko zavarovanje fleks za odrasle


In [ ]:
found_taxonomies =  f"""["Ali je račun poravnan / plačan?", "Ali lahko sklenem nezgodno zavarovanje mladih med letom?", "Blokiran račun, urejanje terjatev", "Do kdaj lahko sklenem zavarovanje odpovedi turističnega zavarovanja?","Investicijsko zavarovanje fleks za odrasle", "i.Triglav nivo zaupanja tier", "Informacije o opominu", "Kaj je digitalna poslovalnica i.triglav?", "Informacije o opominu"]"""

email_question_fact =  f"""Elizabeta Uršič je vprašala, kje najde obrazec za dvig predujma.
keywords -> ["dvig predujma, obrazec, investicijsko zavarovanje, fleks odrasli, št. police 8229870"]"""

prompt_danijel_old = """You are an insurance smart agent in the CRM departmen and works in Zavarovalnica Triglav Slovenia (insurance company).
For client's SUMMARY written on Slovenian language do label classification based on next 10 labels available as JSON list (keys for one label are: "id", "cpxq", "tiid", "value", "rank").
A very important field to match is: "cpxq". Return only one best label match as JSON. Best match needs to return only next keys: "cpxq" (as string), "tiid" (as integer), "value" (as float), "rank" (as integer) and new key "confidence" (as integer between 0 and 100) how sure you are in this label classification/prediction.
Exclude keys: "id".
Labels for Summary: [{'id': 'triglav_c3:499', 'cpxq': 'Zakaj aplikacija Drajv ne beleži pravo število prevoženih kilometrov?\nprijava napake v aplikaciji v Drajv\nKLJUČNE BESEDE: kilometri, km, število prevoženih kilometrov, posodobitev', 'tiid': '500', 'value': '0.0757800936699', 'rank': 1}, {'id': 'triglav_c3:498', 'cpxq': 'Zakaj aplikacija Drajv prikazuje napačno oceno vožnje?\nprijava napake v aplikaciji v Drajv\nKLJUČNE BESEDE: omejitev hitrosti, prometni znaki', 'tiid': '499', 'value': '0.114150226116', 'rank': 2}, {'id': 'triglav_c3:492', 'cpxq': 'Zakaj ne morem začeti snemati vožnje med možnjo?\nUporaba in delovanje aplikacije v Drajv\nPostopek snemanja vožnje in namestitev telefona\nKLJUČNE BESEDE: vključitev aplikacije, snemanje vožnje, drajv', 'tiid': '493', 'value': '0.115358710289', 'rank': 3}, {'id': 'triglav_c3:476', 'cpxq': 'Zakaj aplikacija drajv ne deluje pravilno?\nTehnične težave v Drajv\nKLJUČNE BESEDE: GPS, mobilni podatki, ne deluje, posodobitev, aplikacija', 'tiid': '477', 'value': '0.119953930378', 'rank': 4}, {'id': 'triglav_c3:475', 'cpxq': 'Zakaj se aplikacija ugaša med vožnjo?\ntehnične težave v Drajv\nKLJUČNE BESEDE: vožnja, ugasne aplikacija, Drajv', 'tiid': '476', 'value': '0.124796569347', 'rank': 5}, {'id': 'triglav_c3:501', 'cpxq': 'Kako prijavim merjenje napačnih sil oz. premik telefona?\nprijava napake v aplikaciji v Drajv\nKLJUČNE BESEDE: sile, vožnja, aplikacija, Drajv', 'tiid': '502', 'value': '0.126950263977', 'rank': 6}, {'id': 'triglav_c3:500', 'cpxq': 'Zakaj ne morem prijaviti napake?\nprijava napake v aplikaciji v Drajv\nKLJUČNE BESEDE: prijava, napaka, Drajv', 'tiid': '501', 'value': '0.131350398064', 'rank': 7}, {'id': 'triglav_c3:493', 'cpxq': 'Kakšen je postopek snemanja vožnje?\nUporaba in delovanje aplikacije v Drajv\nPostopek snemanja vožnje in namestitev telefona\nKLJUČNE BESEDE: snemanje vožnje, zaključek, vožnja, stop, drajv', 'tiid': '494', 'value': '0.138219952583', 'rank': 8}, {'id': 'triglav_c3:491', 'cpxq': 'Ali je dovoljeno premikanje telefona med vožnjo?\nUporaba in delovanje aplikacije v Drajv\nPostopek snemanja vožnje in namestitev telefona\nKLJUČNE BESEDE: premik, telefon, ocena, vožnja, drajv', 'tiid': '492', 'value': '0.138740837574', 'rank': 9}, {'id': 'triglav_c3:478', 'cpxq': 'Kako deluje aplikacija Drajv?\nUporaba in delovanje aplikacije v Drajv\nSplošne značilnosti aplikacije\nKLJUČNE BESEDE: aplikacija, snemanje, vožnja, omejitev hitrosti, sile, toleranca', 'tiid': '479', 'value': '0.139850795269', 'rank': 10}]
""" 
prompt_output = gpt4_call(prompt_danijel_old).content

In [26]:
test_prompt_good_acualy_bad = """You are an insurance intelligence agent in the CRM departmen and work at Zavarovalnica Triglav Slovenia (insurance company).
For client's PROBLEM written on Slovenian, do label classification based only on the following 10 labels available as JSON list (keys for one label are: "id", "cpxq", "tiid", "value", "rank").
The most important field to match is: "cpxq". Return only one best label match as JSON.
The best match needs to return only the following keys: "cpxq" (as string), "tiid" (as integer), "value" (as float), "rank" (as integer) and new key "confidence" (as integer between 0 and 100), that tells, how sure you are in this label classification/prediction.
Exclude keys: "id". As a last important step: check if your match exists in Labels for Problem.

Labels for Problem:
[{'id': 'triglav_c3:367', 'cpxq': 'Zakaj sem prejel položnico, če zavarovanja nisem podaljšal?\nAvtomatska obnova zavarovanja v Zavarovalne police\nKLJUČNE BESEDE: avtomatska obnova, zavarovanje dom, zavarovanje hišnih ljubljenčkov, zavarovanje psi, turistično zavarovanje, permanentno zavarovanje', 'tiid': '368', 'value': '0.131021380424', 'rank': 1}, {'id': 'triglav_c3:183', 'cpxq': 'Zakaj sem prejel/a dokumente za obnovo zavarovanja?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje Pazi name\nKLJUČNE BESEDE: dokumenti, obnova zavarovanja, obračun, račun, UPN, nezgodno zavarovanje mladih, permanentna', 'tiid': '184', 'value': '0.147427856922', 'rank': 2}, {'id': 'triglav_c3:155', 'cpxq': 'Kaj lahko dodatno zavarujem?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje otrok in mladine\nKLJUČNE BESEDE: nezgodno zavarovanje, otroci, mladina, kritje, dodatno kritje, doplačilo', 'tiid': '156', 'value': '0.148824572563', 'rank': 3}, {'id': 'triglav_c3:46', 'cpxq': 'Avtomobil imam na lizing. Tudi zavarovanje imam urejeno preko lizing podjetja. Ali to zavarovanje lahko vključim v svoj Triglav komplet?\nTriglav komplet v Terjatve / plačila\nKLJUČNE BESEDE: ', 'tiid': '47', 'value': '0.151032686234', 'rank': 4}, {'id': 'triglav_c3:275', 'cpxq': 'Ali lahko sklenem zavarovanje tudi kadarkoli med letom?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje otrok in mladine\nKLJUČNE BESEDE: sklepanje, nezgodno zavarovanje otrok in mladine, akcijska ponudba', 'tiid': '276', 'value': '0.152361571789', 'rank': 5}, {'id': 'triglav_c3:100', 'cpxq': 'Za polico, ki jo sklepam 2022, se gleda razlika med 2021 in 2020? Vendar v tem primeru, ko sklepam to polico, vozilo še ni opravilo tehničnega pregleda in posledično v eMRLVju ni letošnjega stanja?\nZavarovanja vozil v Zavarovalne police\nZavarovanje avtomobilske odgovornosti (obvezno avtomobilsko zavarovanje)\nKLJUČNE BESEDE: ', 'tiid': '101', 'value': '0.152637004852', 'rank': 6}, {'id': 'triglav_c3:190', 'cpxq': 'Zakaj ne morem skleniti nezgodnega zavarovanja mladih na spletu?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje Pazi name\nKLJUČNE BESEDE: sklenitev preko spleta, online, permanentno zavarovanje, avtomatska obnova, nezgodno zavarovanje mladih', 'tiid': '191', 'value': '0.15369451046', 'rank': 7}, {'id': 'triglav_c3:407', 'cpxq': 'Ali me je kontaktiral vaš zastopnik?\nInformacije glede obiska agenta/zastopnika v Splošne informacije \nKLJUČNE BESEDE: obisk agenta, zastopnika, obisk svetovalca, triglav svetovanje, tris', 'tiid': '408', 'value': '0.155272960663', 'rank': 8}, {'id': 'triglav_c3:112', 'cpxq': 'Ukinja se doplačila za nadomestna vozila – v primeru avto vleke, ki opravlja storitev avto vleke, rabi ali ne to storitev?\nZavarovanja vozil v Zavarovalne police\nZavarovanje avtomobilske odgovornosti (obvezno avtomobilsko zavarovanje)\nKLJUČNE BESEDE: ', 'tiid': '113', 'value': '0.156291544437', 'rank': 9}, {'id': 'triglav_c3:47', 'cpxq': 'Preko spleta smo sklenili letno zavarovanje potovanj v tujino, ki prične veljati šele čez teden dni. Ali zavarovanje lahko vključim v Triglav komplet?\nTriglav komplet v Terjatve / plačila\nKLJUČNE BESEDE: ', 'tiid': '48', 'value': '0.156547427177', 'rank': 10}]

PROBLEM: Klicali so me glede dodatnih storitev, vendar sem pred pol leta zamenjal zavarovalnico za dopolnilno zavarovanje na Generalli in nisem njihova stranka."""


test_prompt_bad = """You are an insurance intelligence agent in the CRM departmen and work at Zavarovalnica Triglav Slovenia (insurance company). For client's PROBLEM written on Slovenian, do label classification based only on the following 10 labels available as JSON list (keys for one label are: "id", "cpxq", "tiid", "value", "rank"). The most important field to match is: "cpxq". Return only one best label match as JSON. The best match needs to return only the following keys: "cpxq" (as string), "tiid" (as integer), "value" (as float), "rank" (as integer) and new key "confidence" (as integer between 0 and 100), that tells, how sure you are in this label classification/prediction. Exclude keys: "id". As a last important step: check if your match exists in Labels for Problem. Labels for Problem: [{'id': 'triglav_c3:367', 'cpxq': 'Zakaj sem prejel položnico, če zavarovanja nisem podaljšal?\nAvtomatska obnova zavarovanja v Zavarovalne police\nKLJUČNE BESEDE: avtomatska obnova, zavarovanje dom, zavarovanje hišnih ljubljenčkov, zavarovanje psi, turistično zavarovanje, permanentno zavarovanje', 'tiid': '368', 'value': '0.131021380424', 'rank': 1}, {'id': 'triglav_c3:183', 'cpxq': 'Zakaj sem prejel/a dokumente za obnovo zavarovanja?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje Pazi name\nKLJUČNE BESEDE: dokumenti, obnova zavarovanja, obračun, račun, UPN, nezgodno zavarovanje mladih, permanentna', 'tiid': '184', 'value': '0.147427856922', 'rank': 2}, {'id': 'triglav_c3:155', 'cpxq': 'Kaj lahko dodatno zavarujem?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje otrok in mladine\nKLJUČNE BESEDE: nezgodno zavarovanje, otroci, mladina, kritje, dodatno kritje, doplačilo', 'tiid': '156', 'value': '0.148824572563', 'rank': 3}, {'id': 'triglav_c3:46', 'cpxq': 'Avtomobil imam na lizing. Tudi zavarovanje imam urejeno preko lizing podjetja. Ali to zavarovanje lahko vključim v svoj Triglav komplet?\nTriglav komplet v Terjatve / plačila\nKLJUČNE BESEDE: ', 'tiid': '47', 'value': '0.151032686234', 'rank': 4}, {'id': 'triglav_c3:275', 'cpxq': 'Ali lahko sklenem zavarovanje tudi kadarkoli med letom?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje otrok in mladine\nKLJUČNE BESEDE: sklepanje, nezgodno zavarovanje otrok in mladine, akcijska ponudba', 'tiid': '276', 'value': '0.152361571789', 'rank': 5}, {'id': 'triglav_c3:100', 'cpxq': 'Za polico, ki jo sklepam 2022, se gleda razlika med 2021 in 2020? Vendar v tem primeru, ko sklepam to polico, vozilo še ni opravilo tehničnega pregleda in posledično v eMRLVju ni letošnjega stanja?\nZavarovanja vozil v Zavarovalne police\nZavarovanje avtomobilske odgovornosti (obvezno avtomobilsko zavarovanje)\nKLJUČNE BESEDE: ', 'tiid': '101', 'value': '0.152637004852', 'rank': 6}, {'id': 'triglav_c3:190', 'cpxq': 'Zakaj ne morem skleniti nezgodnega zavarovanja mladih na spletu?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje Pazi name\nKLJUČNE BESEDE: sklenitev preko spleta, online, permanentno zavarovanje, avtomatska obnova, nezgodno zavarovanje mladih', 'tiid': '191', 'value': '0.15369451046', 'rank': 7}, {'id': 'triglav_c3:407', 'cpxq': 'Ali me je kontaktiral vaš zastopnik?\nInformacije glede obiska agenta/zastopnika v Splošne informacije \nKLJUČNE BESEDE: obisk agenta, zastopnika, obisk svetovalca, triglav svetovanje, tris', 'tiid': '408', 'value': '0.155272960663', 'rank': 8}, {'id': 'triglav_c3:112', 'cpxq': 'Ukinja se doplačila za nadomestna vozila – v primeru avto vleke, ki opravlja storitev avto vleke, rabi ali ne to storitev?\nZavarovanja vozil v Zavarovalne police\nZavarovanje avtomobilske odgovornosti (obvezno avtomobilsko zavarovanje)\nKLJUČNE BESEDE: ', 'tiid': '113', 'value': '0.156291544437', 'rank': 9}, {'id': 'triglav_c3:47', 'cpxq': 'Preko spleta smo sklenili letno zavarovanje potovanj v tujino, ki prične veljati šele čez teden dni. Ali zavarovanje lahko vključim v Triglav komplet?\nTriglav komplet v Terjatve / plačila\nKLJUČNE BESEDE: ', 'tiid': '48', 'value': '0.156547427177', 'rank': 10}] PROBLEM: Klicali so me glede dodatnih storitev, vendar sem pred pol leta zamenjal zavarovalnico za dopolnilno zavarovanje na Generalli in nisem njihova stranka."""


labels_for_problem = """[{'id': 'triglav_c3:367', 'cpxq': 'Informacije glede obiska agenta/zastopnika.\nAvtomatska obnova zavarovanja v Zavarovalne police\nKLJUČNE BESEDE: avtomatska obnova, zavarovanje dom, zavarovanje hišnih ljubljenčkov, zavarovanje psi, turistično zavarovanje, permanentno zavarovanje', 'tiid': '368', 'value': '0.131021380424', 'rank': 1}, {'id': 'triglav_c3:183', 'cpxq': 'Zakaj sem prejel/a dokumente za obnovo zavarovanja?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje Pazi name\nKLJUČNE BESEDE: dokumenti, obnova zavarovanja, obračun, račun, UPN, nezgodno zavarovanje mladih, permanentna', 'tiid': '184', 'value': '0.147427856922', 'rank': 2}, {'id': 'triglav_c3:155', 'cpxq': 'Kaj lahko dodatno zavarujem?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje otrok in mladine\nKLJUČNE BESEDE: nezgodno zavarovanje, otroci, mladina, kritje, dodatno kritje, doplačilo', 'tiid': '156', 'value': '0.148824572563', 'rank': 3}, {'id': 'triglav_c3:46', 'cpxq': 'Avtomobil imam na lizing. Tudi zavarovanje imam urejeno preko lizing podjetja. Ali to zavarovanje lahko vključim v svoj Triglav komplet?\nTriglav komplet v Terjatve / plačila\nKLJUČNE BESEDE: ', 'tiid': '47', 'value': '0.151032686234', 'rank': 4}, {'id': 'triglav_c3:275', 'cpxq': 'Ali lahko sklenem zavarovanje tudi kadarkoli med letom?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje otrok in mladine\nKLJUČNE BESEDE: sklepanje, nezgodno zavarovanje otrok in mladine, akcijska ponudba', 'tiid': '276', 'value': '0.152361571789', 'rank': 5}, {'id': 'triglav_c3:100', 'cpxq': 'Za polico, ki jo sklepam 2022, se gleda razlika med 2021 in 2020? Vendar v tem primeru, ko sklepam to polico, vozilo še ni opravilo tehničnega pregleda in posledično v eMRLVju ni letošnjega stanja?\nZavarovanja vozil v Zavarovalne police\nZavarovanje avtomobilske odgovornosti (obvezno avtomobilsko zavarovanje)\nKLJUČNE BESEDE: ', 'tiid': '101', 'value': '0.152637004852', 'rank': 6}, {'id': 'triglav_c3:190', 'cpxq': 'Zakaj ne morem skleniti nezgodnega zavarovanja mladih na spletu?\nNezgodna zavarovanja v Zavarovalne police\nNezgodno zavarovanje Pazi name\nKLJUČNE BESEDE: sklenitev preko spleta, online, permanentno zavarovanje, avtomatska obnova, nezgodno zavarovanje mladih', 'tiid': '191', 'value': '0.15369451046', 'rank': 7}, {'id': 'triglav_c3:407', 'cpxq': 'Ali me je kontaktiral vaš zastopnik?\nInformacije glede obiska agenta/zastopnika v Splošne informacije \nKLJUČNE BESEDE: obisk agenta, zastopnika, obisk svetovalca, triglav svetovanje, tris', 'tiid': '408', 'value': '0.155272960663', 'rank': 8}, {'id': 'triglav_c3:112', 'cpxq': 'Ukinja se doplačila za nadomestna vozila – v primeru avto vleke, ki opravlja storitev avto vleke, rabi ali ne to storitev?\nZavarovanja vozil v Zavarovalne police\nZavarovanje avtomobilske odgovornosti (obvezno avtomobilsko zavarovanje)\nKLJUČNE BESEDE: ', 'tiid': '113', 'value': '0.156291544437', 'rank': 9}, {'id': 'triglav_c3:47', 'cpxq': 'Preko spleta smo sklenili letno zavarovanje potovanj v tujino, ki prične veljati šele čez teden dni. Ali zavarovanje lahko vključim v Triglav komplet?\nTriglav komplet v Terjatve / plačila\nKLJUČNE BESEDE: ', 'tiid': '48', 'value': '0.156547427177', 'rank': 10}]"""

problem = """Klicali so me glede dodatnih storitev, vendar sem pred pol leta zamenjal zavarovalnico za dopolnilno zavarovanje na Generalli in nisem njihova stranka."""

test_prompt_rewriten = f"""You are an insurance intelligence agent in the CRM department and work at a Slovenian insurance company called Zavarovalnica Triglav.
Consider the following problem (extracted from clients' emails using GPT) and available labels (taxonomies).
From the available taxonomies find the one that best describes the problem.
Available taxonomies are in a JSON list and given in a random order. The most important field you want to match is "cpxq".
Give the output back the same as the JSON in avaliable lables. If the avaliable lables are not in the output, return an JSON that sais "None of the lables match the problem".
------
problem:
{problem}
avaliable lables:
{labels_for_problem}
------
OUTPUT:"""

In [27]:
print(test_prompt_rewriten)

You are an insurance intelligence agent in the CRM department and work at a Slovenian insurance company called Zavarovalnica Triglav.
Consider the following problem (extracted from clients' emails using GPT) and available labels (taxonomies).
From the available taxonomies find the one that best describes the problem.
Available taxonomies are in a JSON list and given in a random order. The most important field you want to match is "cpxq".
Give the output back the same as the JSON in avaliable lables. If the avaliable lables are not in the output, return an JSON that sais "None of the lables match the problem".
------
problem:
Klicali so me glede dodatnih storitev, vendar sem pred pol leta zamenjal zavarovalnico za dopolnilno zavarovanje na Generalli in nisem njihova stranka.
avaliable lables:
[{'id': 'triglav_c3:367', 'cpxq': 'Informacije glede obiska agenta/zastopnika.
Avtomatska obnova zavarovanja v Zavarovalne police
KLJUČNE BESEDE: avtomatska obnova, zavarovanje dom, zavarovanje hiš

In [82]:
prompt_output = gpt4_call(test_prompt_good_acualy_bad).content

In [71]:
print(prompt_output)

{"cpxq": "Kaj lahko dodatno zavarujem?", "tiid": 156, "value": 0.148824572563, "rank": 3, "confidence": 85}
Explanation: The problem statement mentions that the client received a call regarding additional services, but they switched their insurance company for supplementary insurance to Generali half a year ago. The best match for this problem is the label with "cpxq": "Kaj lahko dodatno zavarujem?" as it talks about additional insurance coverage and is related to insurance policies, which is the relevant context for this problem. The confidence of 85 means that the match is fairly certain.


In [28]:
for i in range(6):

    print("-----------------------GPT4---------------------------")
    prompt_output = gpt4_call(test_prompt_rewriten).content
    print(prompt_output)

-----------------------GPT4---------------------------
{"id": "triglav_c3:407", "cpxq": "Ali me je kontaktiral vaš zastopnik?\nInformacije glede obiska agenta/zastopnika v Splošne informacije \nKLJUČNE BESEDE: obisk agenta, zastopnika, obisk svetovalca, triglav svetovanje, tris", "tiid": "408", "value": "0.155272960663", "rank": 8}
-----------------------GPT4---------------------------
{"id": "triglav_c3:407", "cpxq": "Ali me je kontaktiral vaš zastopnik?\nInformacije glede obiska agenta/zastopnika v Splošne informacije \nKLJUČNE BESEDE: obisk agenta, zastopnika, obisk svetovalca, triglav svetovanje, tris", "tiid": "408", "value": "0.155272960663", "rank": 8}
-----------------------GPT4---------------------------
{"id": "triglav_c3:407", "cpxq": "Ali me je kontaktiral vaš zastopnik?\nInformacije glede obiska agenta/zastopnika v Splošne informacije \nKLJUČNE BESEDE: obisk agenta, zastopnika, obisk svetovalca, triglav svetovanje, tris", "tiid": "408", "value": "0.155272960663", "rank": 8

In [18]:
for i in range(6):

    print("-----------------------GPT3---------------------------")
    prompt_output = gpt3_call(test_prompt_rewriten).content
    print(prompt_output)

-----------------------GPT3---------------------------
None of the lables match the problem.
-----------------------GPT3---------------------------
None of the lables match the problem.
-----------------------GPT3---------------------------
None of the labels match the problem.
-----------------------GPT3---------------------------
None of the labels match the problem.
-----------------------GPT3---------------------------
None of the lables match the problem.
-----------------------GPT3---------------------------
None of the lables match the problem.


In [ ]:
keyword_prompt = f"""Consider the following email, keywords genereted from email and a JSON of actual avaliable keywords.
From the avaliable content, find the most relevant keywords from the JSON. The most relevant kewords are the ones that best describes the email and match the generated keywords best. Avaliable keywords are in a JSON format and given in a random order.
------
Email:
{email}
Generated keywords:
{gen_keywords}
Avaliable keywords:
{avaliable_keywords}
------
OUTPUT:
"""